In [ ]:
from datasets import load_from_disk
from peft import LoraConfig
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from trl import SFTTrainer
from trl.trainer import ConstantLengthDataset

In [ ]:
# from huggingface_hub import notebook_login
# notebook_login()

In [ ]:
class Config:
    data_path = "./data/used_car_sales/prepared_car_sales_data"
    # model related
    # model_name = "deepseek-ai/deepseek-llm-7b-base"
    # run_name = "deepseekLLM7bBaseFinetuned"
    # model_name = "openai-community/gpt2-large"
    # run_name = "gpt2_large"
    model_name = "stabilityai/stablelm-3b-4e1t"
    run_name = "stability_stablelm_3b"

    # bitsandbytes params
    load_in_4bit = True
    bnb_4bit_quant_type = "nf4"
    bnb_4bit_use_double_quant = False

    sequence_length = 512

    # Lora params
    r = 4  # the rank of the LoRA matrices
    lora_alpha = 16  # the weight
    lora_dropout = 0.1  # dropout to add to the LoRA layers

    # Training Params
    output_dir = "./car_sales_predictor/"
    evaluation_strategy = "steps"
    max_steps = 1000
    num_train_epochs = 1
    save_strategy = "steps"
    logging_strategy = "steps"
    # eval_steps=0.5
    per_device_train_batch_size = 24
    per_device_eval_batch_size = 8
    learning_rate = 2e-5
    gradient_accumulation_steps = 8
    gradient_checkpointing = True
    weight_decay = 0.01

In [ ]:
cnfg = Config()

## Read data

In [ ]:
dataset = load_from_disk(cnfg.data_path)

In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['input', 'output'],
        num_rows: 446964
    })
    test: Dataset({
        features: ['input', 'output'],
        num_rows: 55871
    })
    valid: Dataset({
        features: ['input', 'output'],
        num_rows: 55870
    })
})

In [ ]:
dataset_train = dataset["train"]
dataset_val = dataset["valid"]
dataset_test = dataset["test"]

In [ ]:
dataset_train.shape

(446964, 2)

## Get Tokenizer

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(cnfg.model_name, padding_side="right")

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
tokenizer.padding_side

'right'

In [ ]:
# tokenizer.padding_side = 'right'

In [ ]:
# tokenizer.padding_side

## Dataloader and Prompt Creation

In [ ]:
def create_promt(row):
    rslt = f"{row['input']} \n ### Instruction:\n Predict the selling price of car as a non-negative number.\n ### Response:\n{row['output']}"
    return rslt

In [ ]:
train_dataset = ConstantLengthDataset(
    tokenizer,
    dataset_train,
    formatting_func=create_promt,
    seq_length=cnfg.sequence_length,
    # infinite=True,
)

In [ ]:
validation_dataset = ConstantLengthDataset(
    tokenizer,
    dataset_val,
    formatting_func=create_promt,
    seq_length=cnfg.sequence_length,
    # infinite=False,
)

## Define Model

In [ ]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [ ]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=cnfg.load_in_4bit,
    bnb_4bit_quant_type=cnfg.bnb_4bit_quant_type,
    bnb_4bit_use_double_quant=cnfg.bnb_4bit_use_double_quant,
)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(
    cnfg.model_name, quantization_config=bnb_config
)

`low_cpu_mem_usage` was None, now set to True since model is quantized.


In [ ]:
print_trainable_parameters(model)

trainable params: 257889280 || all params: 1526666240 || trainable%: 16.89231563802708


## Define Training

In [ ]:
lora_config = LoraConfig(
    r=cnfg.r,  # the rank of the LoRA matrices
    lora_alpha=cnfg.lora_alpha,  # the weight
    lora_dropout=cnfg.lora_dropout,  # dropout to add to the LoRA layers
    bias="none",  # add bias to the nn.Linear layers?
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
    ],  # the name of the layers to add LoRA
    modules_to_save=None,  # layers to unfreeze and train from the original pre-trained model
)

In [ ]:
training_args = TrainingArguments(
    output_dir=cnfg.output_dir,
    evaluation_strategy=cnfg.evaluation_strategy,
    max_steps=cnfg.max_steps,
    save_steps=100,
    logging_steps=100,
    eval_steps=100,
    # num_train_epochs=cnfg.num_train_epochs,
    save_strategy=cnfg.save_strategy,
    logging_strategy=cnfg.logging_strategy,
    per_device_train_batch_size=cnfg.per_device_train_batch_size,
    per_device_eval_batch_size=cnfg.per_device_eval_batch_size,
    learning_rate=cnfg.learning_rate,
    gradient_accumulation_steps=cnfg.gradient_accumulation_steps,
    gradient_checkpointing=cnfg.gradient_checkpointing,
    weight_decay=cnfg.weight_decay,
    run_name=cnfg.run_name,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=validation_dataset,
    peft_config=lora_config,
    packing=True,
    max_seq_length=cnfg.sequence_length,
)

/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
max_steps is given, it will override any value given in num_train_epochs
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:342: UserWarning: You passed `packing=True` to the SFTTrainer, and you are training your model with `max_steps` strategy. The dataset will be iterated until the `max_steps` are reached.
  warnings.warn(


In [ ]:
trainer.train()

`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
100,0.896000,0.501855
200,0.464300,0.443319
300,0.434700,0.428591


/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/home/rohit/Desktop/rohit/virtualenvs/rohit_transformers/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_ree